In [1]:
import numpy as np
from collections import defaultdict

In [2]:
test = ['029A',
        '980A',
        '179A',
        '456A',
        '379A']

# Part 1

In [3]:
key_num = {'7':np.array([0,0]), '8':np.array([0,1]), '9':np.array([0,2]),
           '4':np.array([1,0]), '5':np.array([1,1]), '6':np.array([1,2]),
           '1':np.array([2,0]), '2':np.array([2,1]), '3':np.array([2,2]),
                                '0':np.array([3,1]), 'A':np.array([3,2])}

key_dir = {                     '^':np.array([0,1]), 'A':np.array([0,2]),
           '<':np.array([1,0]), 'v':np.array([1,1]), '>':np.array([1,2])}

class robot():
    def __init__(self, keypad=None, code=None, direction=0):
        self.keypad = keypad
        if self.keypad is None:
            self.position = np.array([0,0])
        else:
            self.position = keypad['A']
            
        self.code = code
        self.inputs = ''
        
        self.direction = direction
        
    def set_keypad(self, keypad):
        self.keypad = keypad
        self.position = keypad['A']
        
    def set_code(self, code):
        self.code = code
        self.inputs = ''
        if self.keypad is not None:
            self.position = self.keypad['A']
            
    def set_direction(self, direction):
        self.direction = direction
            
    def move_lr(self, delta):
        for y in range(0, abs(delta[1])):
            if delta[1] < 0:
                self.inputs += '<'
            else:
                self.inputs += '>'
                
    def move_ud(self, delta):
        for x in range(0, abs(delta[0])):
            if delta[0] < 0:
                self.inputs += '^'
            else:
                self.inputs += 'v'
        
    def make_inputs(self):
        if self.code is None:
            raise Errormessage('No code enetered')
            
        for key in self.code:
            next_position = self.keypad[key]
            delta = next_position - self.position
            
            if '7' in self.keypad:
                if self.position[0] == 3 and next_position[1] == 0:
                    self.move_ud(delta)
                    self.move_lr(delta)
                elif self.position[1] == 0 and next_position[0] == 3:
                    self.move_lr(delta)
                    self.move_ud(delta)
            
                elif delta[1] < 0:
                    self.move_lr(delta)
                    self.move_ud(delta)
                else:
                    self.move_ud(delta)
                    self.move_lr(delta)
                    
            elif '<' in self.keypad:
                if key == '<':
                    self.move_ud(delta)
                    self.move_lr(delta)
                elif np.array_equal(self.position, self.keypad['<']):
                    self.move_lr(delta)
                    self.move_ud(delta)
                    
                elif delta[1] < 0:
                    self.move_lr(delta)
                    self.move_ud(delta)
                else:
                    self.move_ud(delta)
                    self.move_lr(delta)
                            
            self.inputs += 'A'
            
            self.position = next_position
            
    def get_inputs(self):
        if self.inputs == '':
            self.make_inputs()
        return self.inputs
    
    def clear_inputs(self):
        self.inputs = ''
        
def part1(data, part=1):
    if part == 1:
        robots = 2
    else:
        robots = 25
    
    total_score = 0
    #data = [data[-1]]
    robot1 = robot(key_num)
    robot2 = robot(key_dir)
    
    for code in data:
        code = code.strip()
        
        robot1.set_code(code)
        sequence = robot1.get_inputs()
            
        for r in range(0, robots):
            robot2.set_code(sequence)
            sequence = robot2.get_inputs()
                    
        score = len(sequence)
        
        total_score += score*int(code[:-1])
        
    print('Part', part, 'result:', total_score)
        
part1(test, 1)

Part 1 result: 126384


In [4]:
with open('input_day21.txt', 'r') as f:
    data = f.readlines()
    f.close()
    
part1(data, 1)

Part 1 result: 205160


Part 1 will not scale well for Part 2 so...

# Part 2

In [5]:
def get_parts(sequence):
    parts = sequence.split('A')
    parts = parts[:-1]
    for i in range(0, len(parts)):
        parts[i] += 'A'
        
    return parts

def run(data, robots=2):
    part1 = 2
    robots = 25
    
    mapp = {}
    
    total_score = [0,0]
    robot1 = robot(key_num)
    robot2 = robot(key_dir)
    
    for code in data:
        code = code.strip()
        
        robot1.set_code(code)
        sequence = robot1.get_inputs()
        
        parts = get_parts(sequence)
        current_sequence = defaultdict(int)
        for part in parts:
            current_sequence[part] += 1
        
        for r in range(0, robots):
            next_sequence = defaultdict(int)
            
            for key in current_sequence.keys():
                if key in mapp.keys():
                    parts = mapp[key]
                        
                else:
                    robot2.set_code(key)
                    sequence = robot2.get_inputs()
                    parts = get_parts(sequence)
                    
                for part in parts:
                    next_sequence[part] += current_sequence[key]
            
            current_sequence = next_sequence
            
            if r == part1-1:
                score = 0
                for key, value in current_sequence.items():
                    score += len(key)*value
                total_score[0] += score*int(code[:-1])
                    
        score = 0
        for key, value in current_sequence.items():
            score += len(key)*value
        
        total_score[1] += score*int(code[:-1])
        

    print('Part 1 result:', total_score[0])
    print('Part 2 result:', total_score[1])
    
run(test)

Part 1 result: 126384
Part 2 result: 154115708116294


In [6]:
with open('input_day21.txt', 'r') as f:
    data = f.readlines()
    f.close()

run(data)

Part 1 result: 205160
Part 2 result: 252473394928452
